# Homework09

Exercises to practice dimensionality reduction with PCA

## Goals

- Add unsupervised pre-processing to our ML flow: encode -> normalize -> pre-process -> train -> evaluate
- Build an intuition for PCA and when/how to use it
- Understand the benefits of dimensionality reduction

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework.

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/image_utils.py
!wget -q https://github.com/PSAM-5020-2025S-A/Homework07/raw/main/Homework07_utils.pyc

!wget -qO- https://github.com/PSAM-5020-2025S-A/5020-utils/releases/latest/download/0801-500.tar.gz | tar xz

In [ ]:
import PIL.Image as PImage

from os import listdir, path

from data_utils import PCA, RandomForestClassifier

from image_utils import get_pixels, make_image

from Homework07_utils import CamUtils

## PCA and Classification

Let's practice doing some more image classification with `PCA` and analyzing the results.

The dataset is the same one we used in `Homework07`. It has images from $25$ different security cameras, and our task is to separate them by camera. Some of the cameras move, some of them don't, and there are more than $1000$ images, so there's no way we want to do this by hand.

Let's start by loading the training images into a list of pixels, like we did with the `faces` dataset in class.

### Loading Data

If we look at the images in `./data/image/0801-500/train/`, we'll notice that they are named and organized in a slightly different way. They're all in the same directory and the first part of their filename specifies which camera they came from. Even though those `ids` are numbers, they're not sequential, so we'll use some helper functions to extract a unique `label` from their filenames.

This is exactly what the `OrdinalEncoder` class does, but since we only have to encode this one column, we'll do it by hand while we read the files in.

In [ ]:
# this creates a list of all the files in a given directory, that end in .jpg
train_files = [f for f in listdir("./data/image/0801-500/train") if f.endswith(".jpg")]

In [ ]:
# TODO: check and see what is inside the list here
train_files[:10]

Now we'll read the image pixels and extract their labels. `CamUtils.get_label()` is the helper function we'll use to "encode" and return a label id based on the filename.

In [ ]:
pixel_data = []
label_data = []

for fname in train_files:
  label = CamUtils.get_label(fname)
  img = PImage.open(path.join("./data/image/0801-500/train", fname))
  pixel_data.append(get_pixels(img))
  label_data.append(label)

In [ ]:
# TODO: check if labels got extracted correctly by looking at 
#       the first few items of the label list and the filename list
label_data[:10]

Like in `Homework07`, the labels and the filenames won't match exactly since labels start at $0$ and the filenames start at $01$ and skip some numbers.

We can open some images from pixels, just to make sure we loaded them correctly:

In [ ]:
display(make_image(pixel_data[0], width=256))
display(make_image(pixel_data[10], width=256))

### PCA

Create a `DataFrame` called `cam_pca_df` by running `PCA` on the list of pixel data, just like we did in class, and then append a `label` column to it, just like we did in Homewokr07 with: `train_df["label"] = label_data`.

You can change how many components to extract from the images later, but for this first run just use $10$ components so we can compare classification with $10$ `PCA` components to the classification with $10$ random features from `Homework07`.  Make sure to confirm how much of the `variance` is preserved by this transformation.

In [ ]:
# TODO: run PCA, get DataFrame
pca = PCA(n_components=11)
cam_pca_df = pca.fit_transform(pixel_data)

# TODO: add "label" column
cam_pca_df["label"] = label_data

# TODO: print variance
print(pca.explained_variance())

# TODO: take a look at the first 5 rows
cam_pca_df.head(6)


### Interpretation

<span style="color:hotpink">
How many components did you keep ?<br>
Did you try different numbers ?<br>
What percent of variance is kept by this transformation?
</span>

<span style="color:lightgreen;">
- I keept 11 components because I was trying to do achive 80% variance to keep of information in the images. 
</span>

### Reconstruct

Before we go run our classification, let's take a look at some reconstructed images, just to make sure they make sense.

Take a look at reconstructed versions of images $0$, $10$, and whichever other images you might have opened above.

We did this in class. It involves a `inverse_transform()` function.

In [ ]:
## Inverse transform df
pca_pixels = pca.inverse_transform(cam_pca_df)
# TODO: reconstruct image at index 0

display(make_image(pixel_data[15], width=256))
display(make_image(pca_pixels.loc[15], width=256))

# TODO: reconstruct image at index 10
display(make_image(pixel_data[10], width=256))
display(make_image(pca_pixels.loc[10], width=256))

The reconstructed images should look blurrier and noisier than the original ones, but they should still look something like the original ones.

### Classify !

Train a `RandomForestClassifier` on the `PCA` data.

Remember to separate the output feature (`label`) from the input features (`PC0`, `PC1`, etc).

In [ ]:
# TODO: create a classifier
model = RandomForestClassifier()

# TODO: split input and output features
features = cam_pca_df.drop(columns="label")
ids = cam_pca_df["label"]

# TODO: fit the model
model.fit(features, ids)

# TODO: run predictions
train_predictions = model.predict(features)



### Measure Error

For the train set we could use the generic `classification_error()` function that we've used before because we have the true labels for each file, but for measuring accuracy for our `test` set classification, which has secret labels, we have to use a more specific function from the `CamUtils` class.

The function is called `classification_accuracy()` and it takes $2$ parameters, a list of filenames and the predicted labels associated with those filenames.

The result is an accuracy score ratio, between $0.0$ and $1.0$.

We can run this function on the `train` set as well:

In [ ]:
# train_files is the list of files we got way up above
CamUtils.classification_accuracy(train_files, train_predictions)

### Interpretation

<span style="color:hotpink">
What does this mean?<br>
Should we expect the same result from the test dataset?<br>
Why, or why not?
</span>

<span style="color:lightgreen;">
I have a score of 1 which means a perfect fit, which is overfitting. This means that the model knows everything about the data. <br>
The model will most probably perform poorly on the test data beacause of it has overfitting on train data and not general enough on new data. 
</span>

### Run on Test set

Now we'll open the image files inside the `./data/image/0801-500/test/` directory, using almost the exact same steps as we did above to create a `DataFrame` with the exception that we don't have labels for these images.

In [ ]:
# TODO: create a list of files in the test/ directory
TEST_PATH = "/workspaces/Homework09/data/image/0801-500/test"
test_files = [f for f in listdir(TEST_PATH) if f.endswith(".jpg")]

# TODO: check its length and content
len(test_files)

We'll use a similar loop to extract the pixel information from these images, except we don't have `label` data, just pixels.

In [ ]:
test_pixel_data = []

for fname in test_files:
  img = PImage.open(path.join("./data/image/0801-500/test", fname))
  test_pixel_data.append(get_pixels(img))

Now we can run `PCA` and the `RandomForestClassifier`.

Since those are already trained, all we have to do is run their `transform()` function on the data.

In [ ]:
# TODO: run PCA on test pixels, get DataFrame
#       this is the same PCA that was created above, not a new one
test_pca_df = pca.transform(test_pixel_data)
test_pca_df.head()

In [ ]:
test_pca_pixels = pca.inverse_transform(test_pca_df)

It's always a good idea to check the result of the `PCA` transformation by looking at the `DataFrame` and reconstructing some images.

In [ ]:
# TODO: reconstruct one or two images
display(make_image(test_pixel_data[30], width=256))
display(make_image(test_pca_pixels.loc[30], width=256))

# TODO: take a look at the first 5 rows
test_pca_pixels.head(6)

### Interpretation

<span style="color:hotpink"><br>
The reconstructed images from the `test` set will always be noisier than the `train` images.<br><br>
Why?<br><br>
</span>

<span style="color:lightgreen;">

Because the model is traind on other data and the test data is new. The patterns and information from the train data are not exact in the test data. <br>
The photos are more blured and noisy because they have variations in pxiels that are not present in the train dataset and the model can not reconstruct them accuratly.
</span>



Run the classifier and get predicted labels. Then get its accuracy ratio is by running the `CamUtils.classification_accuracy()` function with the list of filenames and list of predictions.

In [ ]:
# TODO: run classifier. This is the same classifier already fitted on the train data above

test_features = test_pca_df
test_predicted = model.predict(test_features)

# TODO: get accuracy
CamUtils.classification_accuracy(test_files, test_predicted)

### Interpretation

<span style="color:hotpink">
What's the accuracy ? How could we improve that if we wanted ?<br>
How does this compare to the classification model from <code>Homework07</code>? They can both be tuned to perform well, but how many features did we need last time in order to get an accuracy comparable to this?
</span>

<span style="color:lightgreen;">
The accuracy is .430 
The model is meaking a righ decision with a rate of 43% which in my opinion a good start as it's not guessing. <br>

To improve the model: 
- Adding more components to train the model on. 
- We can tune our RandomForest classifier. 
- Using another classification algorithm. 

In Homework 07 we did not use PCA to extract the most significant information from the dataset we had. <br>
The classification was trained on actual pixle values and not reduced significant data. <br>

Without PCA, we would need a lot more pixels simply because each pixel contains equal information about the image. <br>
Depending on the dataset, we might need to double the number of features. <br>
In contrast, with PCA, each component applies a linear strategy to capture the most important variation after the previous one. <br>
This means we can add components one by one without worrying about information loss. <br>

</span>